# Algorithm: Planted Noisy kXOR

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `PlantedNoisyKXOR`
Algorithm for Planted Noisy kXOR.

Problem (Problem 2.6 of Ref [1]):

Given a noisy-kXOR instance $\hat{\mathcal{I}}$ which is drawn either:

1. with planted advantage $\rho$, from $\tilde\mathcal{P}^{z}_{n, k}(m, \rho)$.
2. at random, from $\tilde\mathcal{R}_{n, k}(m)$.

output a single bit such that it is whp `1` in case 1, and `0` in case 2.

Algorithm (Section 4.4, Theorem 4.18):
We first split the instance into $\hat{\mathcal{I}} = \mathcal{I} \cup \mathcal{I}_\text{guide}$,
by placing each constraint independently in $\mathcal{I}$ with prob. $1 - \zeta$,
otherwise in $\mathcal{I}_\text{guide}$.
$\zeta$ is picked to be $1 / \ln n$.

#### Parameters
 - `inst_guide`: The subset of contraints $\mathcal{I}_\text{guide}$ for the guided state.
 - `inst_solve`: The subset of constraints $\mathcal{I}$ for eigenvalue estimation.
 - `ell`: Kikuchi parameter $\ell$.
 - `rho`: the planted advantage $\rho$ in the planted case. 

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). 


In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import PlantedNoisyKXOR

### Example Instances

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import KXorInstance
from qualtran.symbolics import HasLength, ln

n, m = sympy.symbols("n m", positive=True, integer=True)
k = sympy.symbols("k", positive=True, integer=True, even=True)
c = sympy.symbols("c", positive=True, integer=True)
ell = c * k
rho = sympy.Symbol(r"\rho", positive=True, real=True)

inst = KXorInstance.symbolic(n, m, k)
zeta = 1 / ln(n)
solve_planted_symbolic = PlantedNoisyKXOR(
    inst_guide=inst.subset(HasLength((1 - zeta) * m)),
    inst_solve=inst.subset(HasLength(zeta * m)),
    ell=ell,
    rho=rho,
)

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import KXorInstance

rng = np.random.default_rng(42)
n, m, k = 50, 1000, 4
ell = k
rho = 0.8

inst = KXorInstance.random_instance(n, m, k, planted_advantage=rho, rng=rng)
solve_planted = PlantedNoisyKXOR.from_inst(inst, ell=ell, rho=rho, zeta=0.1, rng=rng)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([solve_planted_symbolic, solve_planted],
           ['`solve_planted_symbolic`', '`solve_planted`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
solve_planted_symbolic_g, solve_planted_symbolic_sigma = solve_planted_symbolic.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(solve_planted_symbolic_g)
show_counts_sigma(solve_planted_symbolic_sigma)